In [ ]:
import polygraphs as pg
import torch

def generator(epsilon, trials, size):
    # Action B yields Bernoulli payoff of 1 (success)
    # with probability p (= 0.5 + e)
    probs = pg.init.halfs(size) + epsilon
    
    # Number of Bernoulli trials
    count = pg.init.zeros(size) + trials

    # Each node gets a private signal that provides information
    # about whether action B is indeed a good action
    sampler = torch.distributions.binomial.Binomial(total_count=count,
                                                    probs=probs)
    return sampler

# Set random seed
pg.random(123)
epsilon = 0.1
trials = 10
size = (1,)

g = generator(epsilon, trials, size)

# Pick a prior, any value between 0 and 1
params = pg.hyperparameters.InitHyperParameters()
params.constant.value = 0.5
prior = pg.init.constant(size, params)
print("Prior is", prior)

# Get two samples
s1 = g.sample()
s2 = g.sample()
print("1st sample is", s1)
print("2nd sample is", s2)

# At time step 1, update prior using first sample
e1 = pg.ops.math.Evidence(g.logits, s1, g.total_count)

posterior = pg.ops.math.bayes(prior, e1)
print("Posterior( t=1) is", posterior)

# At time step 2, update prior (posterior of previous step) using second sample
e2 = pg.ops.math.Evidence(g.logits, s2, g.total_count)

posterior = pg.ops.math.bayes(posterior, e2)
print("Posterior (t=2) is", posterior)

# At time 1, update prior using both samples
e3 = pg.ops.math.Evidence(g.logits, s1 + s2, 2 * g.total_count)

posterior = pg.ops.math.bayes(prior, e3)
print("Posterior is", posterior)


In [ ]:
# Set random seed
pg.random(123)
epsilon = 0.1
trials = 10
size = (1,)
certainty = 0.6

g = generator(epsilon, trials, size)

# Pick a prior, any value between 0 and 1
params = pg.hyperparameters.InitHyperParameters()
params.constant.value = 0.5
prior = pg.init.constant(size, params)
print("Prior is", prior)

# Get two samples
s1 = g.sample()
s2 = g.sample()
print("1st sample is", s1)
print("2nd sample is", s2)

# At time step 1, update prior using first sample
e1 = pg.ops.math.Evidence(g.logits, s1, g.total_count)

posterior = pg.ops.math.jeffrey(prior, e1, certainty)
print("Posterior( t=1) is", posterior)

# At time step 2, update prior (posterior of previous step) using second sample
e2 = pg.ops.math.Evidence(g.logits, s2, g.total_count)

posterior = pg.ops.math.jeffrey(posterior, e2, certainty)
print("Posterior (t=2) is", posterior)

# At time 1, update prior using both samples
e3 = pg.ops.math.Evidence(g.logits, s1 + s2, 2 * g.total_count)

posterior = pg.ops.math.jeffrey(prior, e3, certainty)
print("Posterior is", posterior)


# Pearl's rule: primer

* There is a prior disease, $d$, probability of 1%.


* In presence of the disease, the likelihood of a positive test $t$ outcome is 90%; in absence of the disease, there is still a chance of 5% that the test comes out positive:

So, what we know is:

* $P(d) = 0.01$
* $P(t|d) = 0.9$ (and, therefore, $P(\neg t|d) = 0.1$) 
* $P(t|\neg d) = 0.05$ (and, therefore, $P(\neg t| \neg d) = 0.95$) 

Apply Bayes' rule:

\begin{align*}
P(d|t) & = \frac{P(t|d)P(d)}{P(t)} \\
& = \frac{P(t|d)P(d)}{P(t|d)P(d) + P(t|\neg d) P(\neg d)} \\
& = \frac{0.9 \cdot 0.01}{0.9 \cdot 0.01 + 0.05 \cdot 0.99} \\
& = \frac{0.009}{0.0585}\\
& \sim 0.15
\end{align*}

\begin{align*}
P(d|\neg t) & = \frac{P(\neg t|d)P(d)}{P(\neg t)} \\
& = \frac{P(\neg t|d)P(d)}{P(\neg t|d)P(d) + P(\neg t|\neg d) P(\neg d)} \\
& = \frac{0.1 \cdot 0.01}{0.1 \cdot 0.01 + 0.95 \cdot 0.99} \\
& = \frac{0.001}{0.9415} \\
& \sim 0.1
\end{align*}

## Pearl's rule:

\begin{align*}
P(t|c) & = \frac{P(c|t)P(t)}{P(c)} \\
& = \frac{P(c|t)P(t)}{P(c|t)P(t) + (P(c|\neg t)P(\neg t)} \\
& = \frac{0.8 \cdot 0.0585}{0.8 \cdot 0.0585 + 0.2 \cdot 0.9415} \\
& = \frac{0.0468}{0.2} = 0.234
\end{align*}

\begin{align*}
P(\neg t|c) & = \frac{P(c|\neg t)P(\neg t)}{P(c)} \\
& = \frac{P(c|\neg t)P(\neg t)}{P(c|\neg t)P(\neg t) + (P(c|t)P(t)} \\
& = \frac{0.2 \cdot 0.9415}{0.8 \cdot 0.0585 + 0.2 \cdot 0.9415} \\
& = \frac{0.1883}{0.2} = 0.9415
\end{align*}